# Ising model using Monte Carlo simulation

### Janos Revesz
### SN: 19111202

## Introduction
In the following notebook I will be using the Ising model from statistical physics to model ferromagnetism. The model is made up of a square lattice with each lattice point containing one spin. The spin is either in the +1/2 or -1/2 position and each spin interacts with it's neighbours, being in a lower energy state when the neighboring spins align.

## Physics
The energy of the lattice is the sum of the lattice points interacting with each other and a possible external magnetic field interacting with the spins. 

The energy due to the spins interacting with each other is due to the quantum mechanical exchange coupling between them:
   $$E_J = -J s_1 s_2$$
  where $s_1$ is the first particle's spin, $s_2$ is the second particle's spin and $ J $ is the quantum mechanical exchange coupling between the spins. Depending on $ J $ the interaction is called ferromagnetism, antiferromagnetism or noninteracting.  
    
  $J>0$ - ferrormagnetic  
  $J<0$ - antiferromagnetic  
  $J=0$ - noninteracting   
    
  When $J>0$ , $ E_j $ is minimised when $s_1$ and $s_2$ have the same sign
  which means that the neighboring spins line up for a lower energy state .  
  When $J<0,E_j $ is minimised when $s_1$ and $s_2$ have the opposite sign so the neighboring spins will be in opposite alignment for the lower energy state.
  
   The energy for the spins interacting with the outside magnetic field is given by:
  $$E_B = -B m s$$
  
 For one particle with spin $ s_1 $ and neighboring spins $ s_2, s_3, s_4, s_5$ the total energy is given by:
 $$ E_{tot} = -J s_1 (s_2+s_3+s_4+s_5) - Bms_1 $$
 
 so for the entire lattice the energy is
 $$E_{tot} = -J \sum_{i,j} s_i s_j - B m \sum_i s_i$$
 where i and j are indexes of lattice sites.
 
 ## Monte Carlo method application to minimise E
   We will be using a Monte Carlo simulation method to minimise the lattice's energy, so find it's equillibrium state for given $J$ and $B$. The method is the following:
   * choose two lattice sites and change their spin alignments
   * if the change in energy of the total system is negative, keep the new alignments and repeat the process
   * if the change in energy is larger than 0 we use the function $p= exp( {-\Delta E/k_b T})$ to determine whether the spins stay in their new elignment or not, this is to allow the system to escape local minimums

In [ ]:
# Appropriate imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### 1. Set up the initial arrangements of spins

In [ ]:
# the number of lattice sites is given by boxlen*boxlen
boxlen = 50
B_over_kT = 0
J_over_kT = 0.5
# initialize the spin array
spins = (-1)**np.random.randint(0,2,size=(boxlen,boxlen))
plt.imshow(spins)
plt.title("Spin alignment of a 50x50 grid")
plt.colorbar(label="Spin")
plt.show()

### 2. Calculate the energy
Calculate the enrgy for the whole lattice using  $$E_{tot} = -J \sum_{i,j} s_i s_j - B m \sum_i s_i$$

In [ ]:
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
            +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
etot = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print("Starting energy is ",etot)

### 3. Write the swap function

The change in energy when swapping the spins of two lattice points is given by calculating the energy change of the spin due to the magnetic fild and the energy change due to the coupling of the spin and it's neighbours.  
The energy of the lattice site before the spin change is $E_{totI} $ where $s_{1i}$ is the initial spin of the site and $s_2+s_3+s_4+s_5$ are the spins of the neighbouring lattice sites. 
$$ E_{totI} = -J s_{1i} (s_2+s_3+s_4+s_5) - Bms_{1i} $$  
The energy of the lattice site after the spin change is $E_{totF} $ where $s_{1f}$ is the spin of the site after the change.  
  
$$ E_{totF} = -J s_{f1} (s_2+s_3+s_4+s_5) - Bms_{f1} $$

$$ \Delta E = E_{totF}-E_{totI} $$ and $$s_{1f} = -s_{1i}$$
  
$$\Delta E = \left(-J s_{f1} (s_2+s_3+s_4+s_5) - Bms_{f1}\right)-\left(-J s_{1i} (s_2+s_3+s_4+s_5) - Bms_{1i}\right) $$
$$\Delta E = \left(J s_{i1} (s_2+s_3+s_4+s_5) + Bms_{i1}\right)+\left(J s_{1i} (s_2+s_3+s_4+s_5) + Bms_{1i}\right) $$
  
$$\Delta E = 2s_{i1} \left(J (s_2+s_3+s_4+s_5) + Bm\right)$$
If we include a second spin $z$ with the same subscripts: 

$$\Delta E = 2s_{i1} \left(J (s_2+s_3+s_4+s_5) + Bm\right)+ 2z_{i1} \left(J (z_2+z_3+z_4+z_5) + Bm\right)$$
  
Note that when the these two lattice sites are neighbouring the energy change due to the two sites coupling is accounted for twice so that has to be extracted once.

In [ ]:
def update_swap(i1,j1,i2,j2):
    """The function receives the coordinates of two lattice sites and changes
    their alignments, if the total energy of the system gets lower doing this
    the spins remain in their new alignment. If the energy increases the probability
    of the spins remaining in their new position is given by p = exp(-deltaE/(k_b T))
    
    Inputs
    
    i1: i coordinate of first lattice point
    j1: j coordinate of first lattice point
    i2: i coordinate of second lattice point
    j2: j coordinate of second lattice point
    
    return: the change of the total energy of the lattice
    """
    # Accounting for the periodic boundary conditions with %boxlen
    i1m1 = (i1-1)%boxlen # i coordinate -1
    i1p1 = (i1+1)%boxlen # i coordinate +1
    j1m1 = (j1-1)%boxlen # j coordinate -1
    j1p1 = (j1+1)%boxlen # j coordinate +1
    sum_neigh_spins1 = spins[i1m1,j1] + spins[i1p1,j1] + spins[i1,j1m1] + spins[i1, j1p1]
    i2m1 = (i2-1)%boxlen #...
    i2p1 = (i2+1)%boxlen
    j2m1 = (j2-1)%boxlen
    j2p1 = (j2+1)%boxlen
    sum_neigh_spins2 = spins[i2m1,j2] + spins[i2p1,j2] + spins[i2,j2m1] + spins[i2, j2p1]
    
    # Check if the spins are neighbours, if yes only account for the energy change once
    if (abs(i1-i2)==0 and abs(j1-j2)==1) or (abs(i1-i2)==1 and abs(j1-j2)==0):
        sum_neigh_spins1-=spins[i2,j2]
    # Calculate the change in energy and the probability positive deltaE
    de = 2*spins[i1,j1]*(J_over_kT*sum_neigh_spins1+ 1/2*B_over_kT)+2*spins[i2,j2]*(J_over_kT*sum_neigh_spins2 + 1/2*B_over_kT)
    p = np.exp(-de)
    if (de<0) or (np.random.random()<p):
        spins[i1,j1] = -spins[i1,j1]
        spins[i2,j2] = -spins[i2,j2]
    else:
        de = 0.0
    return de

### 4. Run the simulation

In [ ]:
# Number of steps to run the simpulation for
Nsteps = 50000
etot = np.zeros(Nsteps+1)
# Accounting for the exchange interaction between the spins
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
# Total energy of the lattice at step 0
etot[0] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print(etot[0])
figIsing = plt.figure(figsize=(10,6))
index = 1
for i in range(Nsteps):
    # Select two points at random to test
    this_i1, this_j1 = (np.random.randint(boxlen), np.random.randint(boxlen))
    this_i2, this_j2 = (np.random.randint(boxlen), np.random.randint(boxlen))
    
    # Calculate the change in E and decide whether the spins remain in their new state or not
    de = update_swap(this_i1,this_j1,this_i2,this_j2)
    sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
    etot[i+1] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
    if i%3000==0:
        ax = figIsing.add_subplot(3,7,index)
        ax.imshow(spins)
        title="step:" + str(i)
        ax.set_title(title)
        index +=1 

### 5. Show total energy


In [ ]:
plt.plot(etot/(J_over_kT*boxlen**2))
plt.title("E of the system over time step")
plt.xlabel("t")
plt.ylabel("E")
plt.show()

In [ ]:
J_over_kT = 1.0
# Repeat simulation
spins = (-1)**np.random.randint(0,2,size=(boxlen,boxlen))

Nsteps = 50000
etot = np.zeros(Nsteps+1)
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
etot[0] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print(etot[0])
figIsing = plt.figure(figsize=(10,6))
index = 1
for i in range(Nsteps):
    # Select two points at random to test
    this_i1, this_j1 = (np.random.randint(boxlen), np.random.randint(boxlen))
    this_i2, this_j2 = (np.random.randint(boxlen), np.random.randint(boxlen))

    de = update_swap(this_i1,this_j1,this_i2,this_j2)
    sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
    etot[i+1] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
    if i%3000==0:
        ax = figIsing.add_subplot(3,7,index)
        ax.imshow(spins)
        title="step:" + str(i)
        ax.set_title(title)
        index +=1 

In [ ]:
plt.plot(etot/(J_over_kT*boxlen**2))
plt.title("E of the system over time step")
plt.xlabel("t")
plt.ylabel("E")
plt.show()

## Conclusion


The energy curve is tending smoother downwards and reaches a smaller minE in the same time steps, and the purple patches are more uniform on the plots. Both of these are due to the exchange interaction between the particle being larger so the neighboring spins get "more aligned". Below there is some further investigation into how different values of J and B effect the spin alignment.

## Including magnetic field in the model
When a strong outside magnetic field is included the exchange interaction between the particles gets negligable and all of the spins tend to line up with the strong outside magnetic field.  

In [ ]:
J_over_kT = 1.0
B_over_kT = 10
# Repeat simulation
spins = (-1)**np.random.randint(0,2,size=(boxlen,boxlen))

Nsteps = 50000
etot = np.zeros(Nsteps+1)
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
etot[0] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print(etot[0])
figIsing = plt.figure(figsize=(10,6))
index = 1
for i in range(Nsteps):
    # Select two points at random to test
    this_i1, this_j1 = (np.random.randint(boxlen), np.random.randint(boxlen))
    this_i2, this_j2 = (np.random.randint(boxlen), np.random.randint(boxlen))

    de = update_swap(this_i1,this_j1,this_i2,this_j2)
    sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
    etot[i+1] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
    if i%3000==0:
        ax = figIsing.add_subplot(3,7,index)
        ax.imshow(spins)
        title="step:" + str(i)
        ax.set_title(title)
        index +=1 

When J and B are the same, the spins still end up almost uniformly aligning up to 1 as the exchange E is minimised when the spins are aligned,and the E due to the magnetic field is minimised when the spins align with it to s=1.

In [ ]:
J_over_kT = 1
B_over_kT = 1
# Repeat simulation
spins = (-1)**np.random.randint(0,2,size=(boxlen,boxlen))

Nsteps = 50000
etot = np.zeros(Nsteps+1)
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
etot[0] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print(etot[0])
figIsing = plt.figure(figsize=(10,6))
index = 1
for i in range(Nsteps):
    # Select two points at random to test
    this_i1, this_j1 = (np.random.randint(boxlen), np.random.randint(boxlen))
    this_i2, this_j2 = (np.random.randint(boxlen), np.random.randint(boxlen))

    de = update_swap(this_i1,this_j1,this_i2,this_j2)
    sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
    etot[i+1] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
    if i%3000==0:
        ax = figIsing.add_subplot(3,7,index)
        ax.imshow(spins)
        title="step:" + str(i)
        ax.set_title(title)
        index +=1 

 ## $J<0$ - antiferromagnetism

When J is negative the spins minimise the energy when they are opposite to each other and even when the magnetic field is forcing the spins to a s=1 state the exchange interaction keeps them oppositely aligned.

In [ ]:
J_over_kT = -1
B_over_kT = 1
# Repeat simulation
spins = (-1)**np.random.randint(0,2,size=(boxlen,boxlen))

Nsteps = 50000
etot = np.zeros(Nsteps+1)
sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
etot[0] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
print(etot[0])
figIsing = plt.figure(figsize=(10,6))
index = 1
for i in range(Nsteps):
    # Select two points at random to test
    this_i1, this_j1 = (np.random.randint(boxlen), np.random.randint(boxlen))
    this_i2, this_j2 = (np.random.randint(boxlen), np.random.randint(boxlen))

    de = update_swap(this_i1,this_j1,this_i2,this_j2)
    sum_neigh_spins = np.roll(spins,1,axis=0)+np.roll(spins,-1,axis=0)\
                +np.roll(spins,1,axis=1)+np.roll(spins,-1,axis=-1)
    etot[i+1] = -np.sum(spins*(B_over_kT + J_over_kT*sum_neigh_spins))
    if i%3000==0:
        ax = figIsing.add_subplot(3,7,index)
        ax.imshow(spins)
        title="step:" + str(i)
        ax.set_title(title)
        index +=1 